In [1]:
from __future__ import print_function, division

import fastai
import fastai.utils
from fastai.fautils import *

import pandas as pd

Using cuDNN version 5110 on context None
Mapped name None to device cuda: Tesla K80 (0000:00:1E.0)
Using Theano backend.


In [2]:
%matplotlib inline
import matplotlib as mlp
import matplotlib.pyplot as plt

In [3]:
full_data_path = os.path.expanduser('~/data/state-farm/')
samp_data_path = os.path.expanduser('~/data/sample-state-farm/')
data_path = samp_data_path
batch_size = 64

# overview

This notebook shows a lot of exploratory work with the State Farm kaggle challenge.  The biggest takeaways are:

1. Find the smallest sample size that produces consistent results
1. Start with very small models and quickly work up in complexity, till you're overfitting
1. Selecting the initial training rate, and adjusting it through training is really important
1. Get familiar with data augmentation, but remember you can't precompute your convolutional layers
1. Dropout is super important, but the value is dependent on your training set size, so you need to relearn it after you finish with the sample set

# create batches

In [4]:
t_batches = get_batches(data_path + 'train', batch_size=batch_size)
v_batches = get_batches(data_path + 'valid', batch_size=2*batch_size, shuffle=False)

(
    val_classes, trn_classes, 
    val_labels, trn_labels, 
    val_filenames, filenames,
    test_filename
) = get_classes(data_path)

Found 1500 images belonging to 10 classes.
Found 1000 images belonging to 10 classes.
Found 1500 images belonging to 10 classes.
Found 1000 images belonging to 10 classes.
Found 0 images belonging to 0 classes.


# linear model

In [5]:
def get_lin_model():

    # starting with BatchNormalization saves us from having to normalize our input manually
    model = Sequential([
        BatchNormalization(axis=1, input_shape=(3, 224, 224)),
        Flatten(),
        Dense(10, activation='softmax')
    ])
    model.compile(
        Adam(), 
        loss='categorical_crossentropy', 
        metrics=['accuracy']
    )
    return model

In [6]:
lm = get_lin_model()
lm.summary()

lm.fit_generator(
    t_batches, 
    t_batches.nb_sample, 
    nb_epoch=1, 
    validation_data=v_batches, 
    nb_val_samples=v_batches.nb_sample
)

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
batchnormalization_1 (BatchNorma (None, 3, 224, 224)   12          batchnormalization_input_1[0][0] 
____________________________________________________________________________________________________
flatten_1 (Flatten)              (None, 150528)        0           batchnormalization_1[0][0]       
____________________________________________________________________________________________________
dense_1 (Dense)                  (None, 10)            1505290     flatten_1[0][0]                  
Total params: 1,505,302
Trainable params: 1,505,296
Non-trainable params: 6
____________________________________________________________________________________________________
Epoch 1/1
1500/1500 [==============================] - 33s - loss: 13.1149 - acc: 0.1413 - val_loss: 13.9727 - val_a

In [7]:
np.round(lm.predict_generator(t_batches, t_batches.nb_sample)[:10],2)

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.]], dtype=float32)

Model is entirely predicting 2 of the classes.  Not very useful.  Lower the learning rate and try again.

In [8]:
lm = get_lin_model()

lm.optimizer.lr.set_value(1e-5)
lm.fit_generator(
    t_batches, 
    t_batches.nb_sample, 
    nb_epoch=2, 
    validation_data=v_batches, 
    nb_val_samples=v_batches.nb_sample
)

Epoch 1/2
1500/1500 [==============================] - 34s - loss: 2.4478 - acc: 0.1987 - val_loss: 3.9639 - val_acc: 0.1620
Epoch 2/2
1500/1500 [==============================] - 27s - loss: 1.8161 - acc: 0.3767 - val_loss: 2.4579 - val_acc: 0.2950


In [9]:
lm.optimizer.lr.set_value(1e-3)
lm.fit_generator(
    t_batches, 
    t_batches.nb_sample, 
    nb_epoch=4,
    validation_data=v_batches, 
    nb_val_samples=v_batches.nb_sample
)

Epoch 1/4
1500/1500 [==============================] - 34s - loss: 9.6578 - acc: 0.3080 - val_loss: 12.8233 - val_acc: 0.1690
Epoch 2/4
1500/1500 [==============================] - 26s - loss: 9.6226 - acc: 0.3627 - val_loss: 9.6096 - val_acc: 0.3770
Epoch 3/4
1500/1500 [==============================] - 26s - loss: 8.8492 - acc: 0.4240 - val_loss: 9.9640 - val_acc: 0.3540
Epoch 4/4
1500/1500 [==============================] - 26s - loss: 8.8242 - acc: 0.4307 - val_loss: 9.1931 - val_acc: 0.4080


Validation accuracy of 0.43 -- almost too good to be true, and definitely better than chance.

Now lets make sure validation set is large enough to ensure a stable accuracy metric across runs, so we aren't making the wrong generalization about the results we're observing.

In [10]:
r_batches = get_batches(data_path+'valid', batch_size=2*batch_size)
val_res = [lm.evaluate_generator(r_batches, r_batches.nb_sample) for i in range(10)]
np.round(val_res, 2)

Found 1000 images belonging to 10 classes.


array([[ 9.19,  0.41],
       [ 9.26,  0.41],
       [ 9.46,  0.39],
       [ 9.08,  0.41],
       [ 9.23,  0.41],
       [ 9.24,  0.4 ],
       [ 9.41,  0.4 ],
       [ 9.04,  0.42],
       [ 9.19,  0.41],
       [ 8.98,  0.42]])

Consisent results.  Accuracy increases larger than 2% can't be attributed to chance.

In [16]:
lm2 = get_lin_model()

rates = [
    (1e-5, 2),
    (1e-4, 4)
]

fastai.utils.fit_generator(lm2, t_batches, rates, val_batches=v_batches)

# lrsched = keras.callbacks.LearningRateScheduler(
#     fastai.utils.list_rate_schedule([
#         (1e-5, 2),
#         (1e-4, 4)
#     ],
#     output=True
#     )
# )

# lm2.fit_generator(
#     t_batches, 
#     t_batches.nb_sample, 
#     nb_epoch=6, 
#     validation_data=v_batches, 
#     nb_val_samples=v_batches.nb_sample,
#     callbacks=[lrsched]
# )

Learning rate: 1e-05
Epoch 1/6
1500/1500 [==============================] - 33s - loss: 2.3704 - acc: 0.2233 - val_loss: 5.2292 - val_acc: 0.1640
Epoch 2/6
1500/1500 [==============================] - 27s - loss: 1.7287 - acc: 0.4407 - val_loss: 3.2239 - val_acc: 0.2150
Learning rate: 0.0001
Epoch 3/6
1500/1500 [==============================] - 26s - loss: 1.2193 - acc: 0.6033 - val_loss: 2.6735 - val_acc: 0.4290
Epoch 4/6
1500/1500 [==============================] - 25s - loss: 0.5838 - acc: 0.8267 - val_loss: 2.2709 - val_acc: 0.4160
Epoch 5/6
1500/1500 [==============================] - 26s - loss: 0.2921 - acc: 0.9127 - val_loss: 2.3365 - val_acc: 0.4500
Epoch 6/6
1500/1500 [==============================] - 26s - loss: 0.1682 - acc: 0.9627 - val_loss: 1.9739 - val_acc: 0.5060


In [11]:
# %run ../fastai/state_farm.py reg-lin-model

# regularized linear model

In [12]:
def get_reglin_model():

    model = Sequential([
        BatchNormalization(axis=1, input_shape=(3, 224, 224)),
        Flatten(),
        Dense(10, activation='softmax', W_regularizer=l2(0.01))
    ])
    model.compile(
        Adam(), 
        loss='categorical_crossentropy', 
        metrics=['accuracy']
    )
    return model

In [14]:
rlm = get_reglin_model()

rates = [
    (1e-5, 2),
    (1e-4, 4)
]

fastai.utils.fit_generator(rlm, t_batches, rates, val_batches=v_batches)

# rlm.optimizer.lr.set_value(1e-5)
# rlm.fit_generator(
#     t_batches, 
#     t_batches.nb_sample, 
#     nb_epoch=2, 
#     validation_data=v_batches, 
#     nb_val_samples=v_batches.nb_sample
# )

# rlm.optimizer.lr.set_value(1e-4)
# rlm.fit_generator(
#     t_batches, 
#     t_batches.nb_sample, 
#     nb_epoch=4, 
#     validation_data=v_batches, 
#     nb_val_samples=v_batches.nb_sample
# )

Epoch 1/2
1500/1500 [==============================] - 35s - loss: 2.5796 - acc: 0.1900 - val_loss: 4.4703 - val_acc: 0.1670
Epoch 2/2
1500/1500 [==============================] - 25s - loss: 1.9313 - acc: 0.4240 - val_loss: 3.6262 - val_acc: 0.2560
Epoch 1/4
1500/1500 [==============================] - 33s - loss: 1.4682 - acc: 0.5953 - val_loss: 4.5419 - val_acc: 0.2700
Epoch 2/4
1500/1500 [==============================] - 25s - loss: 0.7639 - acc: 0.8360 - val_loss: 2.7208 - val_acc: 0.3640
Epoch 3/4
1500/1500 [==============================] - 26s - loss: 0.4586 - acc: 0.9320 - val_loss: 2.1631 - val_acc: 0.5090
Epoch 4/4
1500/1500 [==============================] - 25s - loss: 0.3389 - acc: 0.9787 - val_loss: 2.1320 - val_acc: 0.4410


Single linear model with regularization is getting 45% accuracy.  Dipped at the end which means it's continuing to overfit.

# single dense layer

In [15]:
def get_fc_model():

    model = Sequential([
        BatchNormalization(axis=1, input_shape=(3, 224, 224)),
        Flatten(),
        Dense(100, activation='relu'),
        BatchNormalization(),
        Dense(10, activation='softmax')
    ])
    model.compile(
        Adam(), 
        loss='categorical_crossentropy', 
        metrics=['accuracy']
    )
    return model

In [16]:
fc = get_fc_model()
fc.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
batchnormalization_5 (BatchNorma (None, 3, 224, 224)   12          batchnormalization_input_5[0][0] 
____________________________________________________________________________________________________
flatten_5 (Flatten)              (None, 150528)        0           batchnormalization_5[0][0]       
____________________________________________________________________________________________________
dense_5 (Dense)                  (None, 100)           15052900    flatten_5[0][0]                  
____________________________________________________________________________________________________
batchnormalization_6 (BatchNorma (None, 100)           400         dense_5[0][0]                    
___________________________________________________________________________________________

In [17]:
rates = [
    (1e-5, 2),
    (1e-2, 5)
]

fastai.utils.fit_generator(fc, t_batches, rates, val_batches=v_batches)

# fc.optimizer.lr.set_value(1e-5)
# fc.fit_generator(
#     t_batches, 
#     t_batches.nb_sample, 
#     nb_epoch=2, 
#     validation_data=v_batches, 
#     nb_val_samples=v_batches.nb_sample
# )

# fc.optimizer.lr.set_value(0.01)
# fc.fit_generator(
#     t_batches, 
#     t_batches.nb_sample, 
#     nb_epoch=5, 
#     validation_data=v_batches, 
#     nb_val_samples=v_batches.nb_sample
# )

Epoch 1/2
1500/1500 [==============================] - 39s - loss: 2.0811 - acc: 0.3260 - val_loss: 4.4430 - val_acc: 0.2200
Epoch 2/2
1500/1500 [==============================] - 31s - loss: 1.1320 - acc: 0.6967 - val_loss: 3.5516 - val_acc: 0.2420
Epoch 1/5
1500/1500 [==============================] - 34s - loss: 1.9149 - acc: 0.3753 - val_loss: 11.7057 - val_acc: 0.0810
Epoch 2/5
1500/1500 [==============================] - 25s - loss: 1.3464 - acc: 0.5887 - val_loss: 8.8302 - val_acc: 0.1290
Epoch 3/5
1500/1500 [==============================] - 26s - loss: 0.9059 - acc: 0.7620 - val_loss: 5.9977 - val_acc: 0.1590
Epoch 4/5
1500/1500 [==============================] - 26s - loss: 0.5755 - acc: 0.8487 - val_loss: 4.2873 - val_acc: 0.2830
Epoch 5/5
1500/1500 [==============================] - 25s - loss: 0.3329 - acc: 0.9273 - val_loss: 3.5137 - val_acc: 0.3470


Beginning to learn the training set, but failing at validation.

# simple conv layers

In [17]:
def get_conv_model(t_batches=t_batches, v_batches=v_batches, train=True):

    model = Sequential([
        BatchNormalization(axis=1, input_shape=(3, 224, 224)),
        Convolution2D(32,3,3, activation='relu'),
        BatchNormalization(axis=1),
        MaxPooling2D((3,3)),
        Convolution2D(64,3,3, activation='relu'),
        BatchNormalization(axis=1),
        MaxPooling2D((3,3)),
        Flatten(),
        Dense(200, activation='relu'),
        BatchNormalization(),
        Dense(10, activation='softmax')
    ])
    model.compile(
        Adam(), 
        loss='categorical_crossentropy', 
        metrics=['accuracy']
    )
    
    if not train:
        return model
    
    rates = [
        (1e-4, 2),
        (1e-3, 4)
    ]

    fastai.utils.fit_generator(model, t_batches, rates, val_batches=v_batches)
    
#     model.optimizer.lr.set_value(1e-4)
#     h = model.fit_generator(
#         t_batches, 
#         t_batches.nb_sample, 
#         nb_epoch=2, 
#         validation_data=v_batches, 
#         nb_val_samples=v_batches.nb_sample
#     )

#     model.optimizer.lr.set_value(1e-3)
#     h = model.fit_generator(
#         t_batches, 
#         t_batches.nb_sample, 
#         nb_epoch=4, 
#         validation_data=v_batches, 
#         nb_val_samples=v_batches.nb_sample
#     )
    
    return model

In [20]:
cm = get_conv_model()
# cm.summary()

Epoch 1/2
1500/1500 [==============================] - 38s - loss: 1.4453 - acc: 0.5767 - val_loss: 2.1388 - val_acc: 0.2220
Epoch 2/2
1500/1500 [==============================] - 28s - loss: 0.3380 - acc: 0.9327 - val_loss: 2.0896 - val_acc: 0.2450
Epoch 1/4
1500/1500 [==============================] - 35s - loss: 0.2581 - acc: 0.9393 - val_loss: 1.8649 - val_acc: 0.3250
Epoch 2/4
1500/1500 [==============================] - 27s - loss: 0.0760 - acc: 0.9893 - val_loss: 2.1105 - val_acc: 0.2950
Epoch 3/4
1500/1500 [==============================] - 29s - loss: 0.0178 - acc: 0.9987 - val_loss: 2.4252 - val_acc: 0.2910
Epoch 4/4
1500/1500 [==============================] - 26s - loss: 0.0051 - acc: 1.0000 - val_loss: 2.6336 - val_acc: 0.2280


Very rapidly learning the training data and failing to generalize to the validation dataset.

# data augmentation

In [21]:
gen_t = image.ImageDataGenerator(width_shift_range=0.1)
batches = get_batches(data_path+'train', gen_t, batch_size=batch_size)

model = get_conv_model(batches)

Found 1500 images belonging to 10 classes.
Epoch 1/2
1500/1500 [==============================] - 33s - loss: 2.1154 - acc: 0.3207 - val_loss: 1.9274 - val_acc: 0.2960
Epoch 2/2
1500/1500 [==============================] - 27s - loss: 1.1553 - acc: 0.6333 - val_loss: 2.2009 - val_acc: 0.1880
Epoch 1/4
1500/1500 [==============================] - 38s - loss: 1.1049 - acc: 0.6620 - val_loss: 1.9074 - val_acc: 0.3850
Epoch 2/4
1500/1500 [==============================] - 27s - loss: 0.6780 - acc: 0.7940 - val_loss: 2.0739 - val_acc: 0.3630
Epoch 3/4
1500/1500 [==============================] - 26s - loss: 0.4214 - acc: 0.8820 - val_loss: 2.2804 - val_acc: 0.3310
Epoch 4/4
1500/1500 [==============================] - 26s - loss: 0.3258 - acc: 0.8993 - val_loss: 2.7369 - val_acc: 0.2650


In [22]:
gen_t = image.ImageDataGenerator(height_shift_range=0.05)
batches = get_batches(data_path+'train', gen_t, batch_size=batch_size)

model = get_conv_model(batches)

Found 1500 images belonging to 10 classes.
Epoch 1/2
1500/1500 [==============================] - 34s - loss: 1.9176 - acc: 0.3973 - val_loss: 2.2439 - val_acc: 0.2740
Epoch 2/2
1500/1500 [==============================] - 28s - loss: 0.7708 - acc: 0.7860 - val_loss: 1.9366 - val_acc: 0.2760
Epoch 1/4
1500/1500 [==============================] - 35s - loss: 0.7150 - acc: 0.7840 - val_loss: 1.9357 - val_acc: 0.3570
Epoch 2/4
1500/1500 [==============================] - 26s - loss: 0.3415 - acc: 0.9013 - val_loss: 1.8168 - val_acc: 0.3980
Epoch 3/4
1500/1500 [==============================] - 27s - loss: 0.1738 - acc: 0.9573 - val_loss: 2.3124 - val_acc: 0.2000
Epoch 4/4
1500/1500 [==============================] - 29s - loss: 0.1065 - acc: 0.9740 - val_loss: 2.8378 - val_acc: 0.1480


In [23]:
gen_t = image.ImageDataGenerator(shear_range=0.1)
batches = get_batches(data_path+'train', gen_t, batch_size=batch_size)

model = get_conv_model(batches)

Found 1500 images belonging to 10 classes.
Epoch 1/2
1500/1500 [==============================] - 36s - loss: 1.7012 - acc: 0.4827 - val_loss: 1.9316 - val_acc: 0.2870
Epoch 2/2
1500/1500 [==============================] - 26s - loss: 0.4458 - acc: 0.8960 - val_loss: 2.0243 - val_acc: 0.2620
Epoch 1/4
1500/1500 [==============================] - 34s - loss: 0.4238 - acc: 0.8813 - val_loss: 1.9964 - val_acc: 0.1850
Epoch 2/4
1500/1500 [==============================] - 26s - loss: 0.1763 - acc: 0.9640 - val_loss: 1.7337 - val_acc: 0.4380
Epoch 3/4
1500/1500 [==============================] - 28s - loss: 0.0602 - acc: 0.9873 - val_loss: 1.7106 - val_acc: 0.4890
Epoch 4/4
1500/1500 [==============================] - 25s - loss: 0.0288 - acc: 0.9953 - val_loss: 1.7242 - val_acc: 0.4750


In [24]:
gen_t = image.ImageDataGenerator(rotation_range=15)
batches = get_batches(data_path+'train', gen_t, batch_size=batch_size)

model = get_conv_model(batches)

Found 1500 images belonging to 10 classes.
Epoch 1/2
1500/1500 [==============================] - 38s - loss: 2.1082 - acc: 0.3393 - val_loss: 2.7930 - val_acc: 0.1790
Epoch 2/2
1500/1500 [==============================] - 26s - loss: 0.9799 - acc: 0.6927 - val_loss: 2.0522 - val_acc: 0.2930
Epoch 1/4
1500/1500 [==============================] - 34s - loss: 0.9304 - acc: 0.7147 - val_loss: 2.4012 - val_acc: 0.3160
Epoch 2/4
1500/1500 [==============================] - 26s - loss: 0.5048 - acc: 0.8600 - val_loss: 1.9779 - val_acc: 0.3050
Epoch 3/4
1500/1500 [==============================] - 27s - loss: 0.3288 - acc: 0.9100 - val_loss: 1.8143 - val_acc: 0.3710
Epoch 4/4
1500/1500 [==============================] - 26s - loss: 0.2417 - acc: 0.9307 - val_loss: 1.6527 - val_acc: 0.3770


In [25]:
gen_t = image.ImageDataGenerator(channel_shift_range=20)
batches = get_batches(data_path+'train', gen_t, batch_size=batch_size)

model = get_conv_model(batches)

Found 1500 images belonging to 10 classes.
Epoch 1/2
1500/1500 [==============================] - 34s - loss: 1.6338 - acc: 0.5053 - val_loss: 1.9960 - val_acc: 0.2460
Epoch 2/2
1500/1500 [==============================] - 28s - loss: 0.3635 - acc: 0.9240 - val_loss: 1.7223 - val_acc: 0.3640
Epoch 1/4
1500/1500 [==============================] - 35s - loss: 0.2395 - acc: 0.9400 - val_loss: 1.7038 - val_acc: 0.4230
Epoch 2/4
1500/1500 [==============================] - 30s - loss: 0.1028 - acc: 0.9807 - val_loss: 1.7499 - val_acc: 0.3690
Epoch 3/4
1500/1500 [==============================] - 30s - loss: 0.0247 - acc: 0.9980 - val_loss: 1.8515 - val_acc: 0.4990
Epoch 4/4
1500/1500 [==============================] - 26s - loss: 0.0074 - acc: 1.0000 - val_loss: 1.9942 - val_acc: 0.4740


# all together

In [18]:
gen_t = image.ImageDataGenerator(
    width_shift_range=0.1,
    height_shift_range=0.05,
    shear_range=0.1,
    rotation_range=15,
    channel_shift_range=20
)
batches = get_batches(data_path + 'train', gen_t, batch_size=batch_size)

model = get_conv_model(batches)

Found 1500 images belonging to 10 classes.


KeyboardInterrupt: 

In [ ]:
rates = [
    (1e-4, 30),
]

fastai.utils.fit_generator(model, t_batches, rates, val_batches=v_batches)

# model.optimizer.lr.set_value(0.0001)
# model.fit_generator(
#     batches, batches.nb_sample, 
#     nb_epoch=5, 
#     validation_data=v_batches, nb_val_samples=v_batches.nb_sample)
# model.fit_generator(
#     batches, batches.nb_sample, 
#     nb_epoch=25, 
#     validation_data=v_batches, nb_val_samples=v_batches.nb_sample)

In [31]:
vf_batches = get_batches(full_data_path + 'valid', batch_size=2*batch_size, shuffle=False)
model.evaluate_generator(vf_batches, vf_batches.nb_sample)

Found 2109 images belonging to 10 classes.


[1.0532610309016597, 0.71692745405217939]

In [29]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
batchnormalization_31 (BatchNorm (None, 3, 224, 224)   12          batchnormalization_input_12[0][0]
____________________________________________________________________________________________________
convolution2d_13 (Convolution2D) (None, 32, 222, 222)  896         batchnormalization_31[0][0]      
____________________________________________________________________________________________________
batchnormalization_32 (BatchNorm (None, 32, 222, 222)  128         convolution2d_13[0][0]           
____________________________________________________________________________________________________
maxpooling2d_13 (MaxPooling2D)   (None, 32, 74, 74)    0           batchnormalization_32[0][0]      
___________________________________________________________________________________________

In [30]:
model.save(data_path + 'state-farm-cnn.h5')

# reload model

# full training plot

In [32]:
# history = []

# gen_t = image.ImageDataGenerator(
#     width_shift_range=0.1,
#     height_shift_range=0.05,
#     shear_range=0.1,
#     rotation_range=15,
#     channel_shift_range=20
# )
# batches = get_batches(data_path + 'train', gen_t, batch_size=batch_size)

# model = get_conv_model(train=False)
# model.optimizer.lr.set_value(1e-4)
# h = model.fit_generator(
#     batches, 
#     batches.nb_sample, 
#     nb_epoch=2, 
#     validation_data=v_batches, 
#     nb_val_samples=v_batches.nb_sample
# )
# history.append(h)

# model.optimizer.lr.set_value(1e-3)
# h = model.fit_generator(
#     batches, 
#     batches.nb_sample, 
#     nb_epoch=4, 
#     validation_data=v_batches, 
#     nb_val_samples=v_batches.nb_sample
# )
# history.append(h)

In [33]:
# model.optimizer.lr.set_value(0.0001)
# h = model.fit_generator(
#     batches, batches.nb_sample, 
#     nb_epoch=5, 
#     validation_data=v_batches, nb_val_samples=v_batches.nb_sample)
# history.append(h)
# h = model.fit_generator(
#     batches, batches.nb_sample, 
#     nb_epoch=25, 
#     validation_data=v_batches, nb_val_samples=v_batches.nb_sample)
# history.append(h)

In [34]:
# acc = []
# val_acc = []
# for h in history:
#     acc += h.history['acc']
#     val_acc += h.history['val_acc']
    
# plt.plot(acc)
# plt.plot(val_acc)

# submit

Compute test set output and actually submit.

In [4]:
from fastai import kaggle

def submission_df(preds, test_batches, classes):
    # construct dataframe of the submission
    index = pd.Series(
        [f.split('/')[-1] for f in test_batches.filenames],
        name='img'
    )

    df = pd.DataFrame(
        preds,
        index=index,
        columns=classes
    )

    return df.sort_index()

In [5]:
test_batches = get_batches(
    full_data_path + 'test', shuffle=False, batch_size=batch_size * 2,
    class_mode=None)
train_batches = get_batches(
    full_data_path + 'train', shuffle=False, batch_size=batch_size,
    class_mode=None)
classes = sorted(train_batches.class_indices)

Found 79726 images belonging to 1 classes.
Found 20315 images belonging to 10 classes.


In [ ]:
model = keras.models.load_model(samp_data_path + 'state-farm-cnn.h5')

In [10]:
# predict
preds = model.predict_generator(test_batches, test_batches.nb_sample)

KeyboardInterrupt: 

In [ ]:
df = submission_df(preds, test_batches, classes)
df = df.clip(0.05, 0.95)
df.to_csv(full_data_path + 'submission.csv', index=True)

In [ ]:
import subprocess

cmd = [
    'kg',
    'submit',
    '-u', os.environ['KAGGLE_USERNAME'],
    '-p', os.environ['KAGGLE_PASSWORD'],
    '-c', 'state-farm-distracted-driver-detection',
    full_data_path + 'submission.csv'
]

subprocess.call(cmd)